# Power System Datasets Preparation

This script generates the four core datasets used in power system analysis: Buses, Generators, Lines and Loads.

## Buses

In [2]:
import pandas as pd

df_Buses = pd.read_csv('./prerun/buses_GB.csv')
df_Buses.rename(columns={'bus_id': 'name'}, inplace=True)
df_Buses.insert(0, 'BusID', range(1, len(df_Buses) + 1))
df_Buses = df_Buses.loc[:, :'y']
df_Buses = df_Buses.drop(columns=['dc', 'symbol', 'under_construction', 'tags'])

df_Buses

,BusID,name,voltage,x,y
0,1,GB1-220,220,-2.895746,58.256815
1,2,GB10-275,275,-3.719714,56.088133
2,3,GB11-275,275,-3.179692,55.924620
3,4,GB12-275,275,-3.178627,55.903134
4,5,GB13-400,400,-2.938918,55.884694
...,...,...,...,...,...
496,497,way/97551523-400,400,-2.416744,55.948838
497,498,way/97561542-275,275,-1.116723,54.606500
498,499,way/975637991,320,-1.271945,60.293401
499,500,way/98788846-275,275,-3.130160,51.481960


### Asign them into regions

In [3]:
regions = pd.read_csv('./GBPower_raw/gsp_gnode_directconnect_region_lookup.csv')
regions

,ng_id,ggd_id,gnode_id,gnode_name,gnode_lat,gnode_lon,gsp_id,gsp_name,gsp_lat,gsp_lon,dc_id,dc_name,dc_lat,dc_lon,region_id,region_name,has_pv,pes_id,pes_name
0,1,1,1.0,COWL,51.71105,-1.18901,1.0,COWL_1,51.711050,-1.189010,NaN,NaN,NaN,NaN,1.0,Cowley,1,20.0,_H
1,2,2,2.0,ECLA,51.92579,-0.90565,2.0,ECLA_1,51.925790,-0.905650,NaN,NaN,NaN,NaN,19.0,East Claydon,1,11.0,_B
2,3,3,3.0,ENDEDW,52.59726,-1.21548,3.0,ENDE_1,52.597260,-1.215480,NaN,NaN,NaN,NaN,20.0,Enderby,1,11.0,_B
3,4,4,4.0,GRENDW,52.24269,-0.72736,4.0,GREN_1,52.242690,-0.727360,NaN,NaN,NaN,NaN,21.0,Grendon,1,11.0,_B
4,5,5,5.0,RATS,52.86342,-1.25444,5.0,RATS_1,52.863420,-1.254440,NaN,NaN,NaN,NaN,22.0,Ratcliffe,1,11.0,_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,376,376,360.0,BPGR,56.01141,-3.72183,NaN,NaN,NaN,NaN,17.0,T_BPGRD-2,56.01141,-3.72183,NaN,NaN,0,NaN,NaN
376,377,377,NaN,NaN,NaN,NaN,335.0,KNAR,54.031211,-1.487844,NaN,NaN,NaN,NaN,132.0,Knaresborough,1,15.0,_F
377,378,378,NaN,NaN,NaN,NaN,336.0,WYLF_1,53.413644,-4.482279,NaN,NaN,NaN,NaN,114.0,Wylfa,1,13.0,_D
378,379,379,NaN,NaN,NaN,NaN,337.0,DALM3,55.838567,-4.208346,NaN,NaN,NaN,NaN,260.0,Dalmarnock,1,18.0,_N


### Load the data

In [4]:
import geopandas as gpd
from shapely.geometry import Point

gdf_regions = gpd.read_file("./GBPower_raw/gsp_geometries.geojson")
gdf_regions = gdf_regions.to_crs("EPSG:4326")
gdf_buses = gpd.GeoDataFrame(
    df_Buses,
    geometry=gpd.points_from_xy(df_Buses['x'], df_Buses['y']),
    crs="EPSG:4326"
)

gdf_regions = gdf_regions[gdf_regions.is_valid]

### Match Buses into regions

In [5]:
# Match each bus to the region polygon it falls within
gdf_joined = gpd.sjoin(gdf_buses, gdf_regions, how="left", predicate="within")

cols_to_keep = ['BusID', 'name', 'x', 'y', 'RegionName']
gdf_joined[cols_to_keep]

,BusID,name,x,y,RegionName
0,1,GB1-220,-2.895746,58.256815,NaN
1,2,GB10-275,-3.719714,56.088133,Devonside
2,3,GB11-275,-3.179692,55.924620,Whitehouse
3,4,GB12-275,-3.178627,55.903134,Kaimes
4,5,GB13-400,-2.938918,55.884694,Cockenzie
...,...,...,...,...,...
496,497,way/97551523-400,-2.416744,55.948838,Dunbar
497,498,way/97561542-275,-1.116723,54.606500,Lackenby
498,499,way/975637991,-1.271945,60.293401,Fraserburgh
499,500,way/98788846-275,-3.130160,51.481960,Cardiff East


In [6]:
df_Buses['RegionName'] = gdf_joined['RegionName']
df_Buses

,BusID,name,voltage,x,y,RegionName
0,1,GB1-220,220,-2.895746,58.256815,NaN
1,2,GB10-275,275,-3.719714,56.088133,Devonside
2,3,GB11-275,275,-3.179692,55.924620,Whitehouse
3,4,GB12-275,275,-3.178627,55.903134,Kaimes
4,5,GB13-400,400,-2.938918,55.884694,Cockenzie
...,...,...,...,...,...,...
496,497,way/97551523-400,400,-2.416744,55.948838,Dunbar
497,498,way/97561542-275,275,-1.116723,54.606500,Lackenby
498,499,way/975637991,320,-1.271945,60.293401,Fraserburgh
499,500,way/98788846-275,275,-3.130160,51.481960,Cardiff East


### Asign them into different colors

In [ ]:
Buses_combined = pd.read_csv('./prerun/buses_mapping/buses_combined.csv',dtype = {'bus385_id': str})
bus_501_385_mapping = dict(zip(Buses_combined['bus501_id'], Buses_combined['bus385_id']))
bus_501_385_mapping

'8575'

In [ ]:
df_Buses['color'] = df_Buses['name'].map(bus_501_385_mapping).apply(
    lambda x: 'green' if pd.notna(x) else 'red'
)
df_Buses.rename(columns={'name': 'Bus name'}, inplace=True)
df_Buses.to_csv('Buses.csv', index=False)

## Loads

In [68]:
df_Loads = pd.read_csv('./GBPower_raw/load_weights.csv')
df_Loads.insert(0, 'LoadID', range(1, len(df_Loads) + 1))
df_Loads

,LoadID,name,load_weight
0,1,8838,0.002569
1,2,8013,0.000261
2,3,8520,0.000241
3,4,8649,0.004041
4,5,4950,0.003187
...,...,...,...
380,381,7458,0.001419
381,382,8283,0.005545
382,383,8094,0.004038
383,384,8093,0.004038


In [69]:
df_bus385_to_501 = pd.read_csv('./prerun/bus385_to_501.csv', dtype={'bus385_id': str})
df_bus385_to_501

,bus385_id,bus501_id
0,6441,relation/8087295-275
1,6442,way/495624575-275
2,6443,way/1262971878
3,6447,way/568527671
4,6449,way/1185489725-220
...,...,...
380,8016,way/26938722-400
381,5210,way/26938722-400
382,5208,way/34656007-400
383,5207,relation/9572254-400


In [70]:
map_385_to_501 = dict(zip(df_bus385_to_501['bus385_id'], df_bus385_to_501['bus501_id']))

df_Loads['name'] = df_Loads['name'].astype(str)
df_Loads['Bus name'] = df_Loads['name'].map(map_385_to_501)

df_Loads['name'] = df_Loads['Bus name']

df_Loads.drop(columns=['Bus name'], inplace=True)
df_Loads.rename(columns={'name': 'Bus name'}, inplace=True)
df_Loads

,LoadID,Bus name,load_weight
0,1,way/92419253-275,0.002569
1,2,way/1190395478-220,0.000261
2,3,way/262523325-400,0.000241
3,4,way/49499923-400,0.004041
4,5,way/25935453-400,0.003187
...,...,...,...
380,381,GB71-275,0.001419
381,382,GB52-400,0.005545
382,383,way/87464485-400,0.004038
383,384,way/87464485-400,0.004038


In [74]:
Bus_region_map = dict(zip(df_Buses['name'], df_Buses['RegionName']))
df_Loads['RegionName'] = df_Loads['Bus name'].map(Bus_region_map)
df_Loads.to_csv('Loads.csv', index=False)

## Generators

In [98]:
df_Units = pd.read_csv('./prerun/units.csv')
df_Units.insert(0, 'UnitID', range(1, len(df_Units) + 1))
df_Units.rename(columns={'NationalGridBmUnit': 'name','carrier': 'type', 'bus': 'Bus name' ,'lon': 'x', 'lat': 'y'}, inplace=True)
df_Units.to_csv('Units.csv', index=False)

## Lines

In [85]:
df_Lines = pd.read_csv('./prerun/lines_GB.csv')
df_Lines.insert(0, 'LineID', range(1, len(df_Lines) + 1))
cols_to_keep = ['LineID', 'bus0', 'bus1', 'voltage', 's_nom', 'r', 'x', 'b', 'length', 'under_construction']
df_Lines = df_Lines[cols_to_keep]
df_Lines.to_csv('Lines.csv', index=False)
df_Lines

,LineID,bus0,bus1,voltage,s_nom,r,x,b,length,under_construction
0,1,way/1307462736-275,way/86071326-275,275,921.668,2.200334,14.577213,0.000228,55008.35,f
1,2,relation/7881280-400,relation/13110369-400,400,3574.953,0.420393,3.447219,0.000243,28026.17,f
2,3,way/101077623-275,GB41-275,275,1843.335,0.046871,0.310518,0.000019,2343.53,f
3,4,way/95279334-400,way/103915328-400,400,3574.953,0.425859,3.492045,0.000246,28390.61,f
4,5,way/111528318-220,GB8-220,220,1474.668,1.584632,7.949573,0.000933,79231.62,f
...,...,...,...,...,...,...,...,...,...,...
585,586,relation/13110369-400,way/314338790-400,400,3574.953,0.103282,0.846912,0.000060,6885.46,f
586,587,relation/7879831-400,way/95749833-400,400,3574.953,0.929985,7.625876,0.000538,61998.99,f
587,588,way/96067520-275,GB59-275,275,1843.335,0.214388,1.420323,0.000089,10719.42,f
588,589,GB61-275,way/96067520-275,275,1843.335,0.173507,1.149483,0.000072,8675.34,f


## Links

In [92]:
df_Links = pd.read_csv('./prerun/links_GB.csv')
df_Links.insert(0, 'LinkID', range(1, len(df_Links) + 1))
df_Links.rename(columns={'link_id': 'name'}, inplace=True)
cols_to_keep = ['LinkID', 'name', 'bus0', 'bus1', 'voltage', 'p_nom', 'length', 'under_construction']
df_Links = df_Links[cols_to_keep]
df_Links.to_csv('Links.csv', index=False)
df_Links

,LinkID,name,bus0,bus1,voltage,p_nom,length,under_construction
0,1,relation/10377412-320-DC,way/753113423,way/636630368,320,1000,230137.18,f
1,2,relation/13295785-515-DC,relation/12832053,way/642490160,515,1400,723362.06,f
2,3,relation/14126301-450-DC,way/109189896,way/920127890,450,1000,253393.42,f
3,4,relation/15772117-320-DC,way/775577829,way/775577827,320,1000,51576.89,f
4,5,relation/15775538-600-DC,way/297840585,way/375626500,600,2250,397643.50,f
5,6,relation/15777152-320-DC,way/1262971878,way/975637991,320,600,282108.95,f
6,7,relation/15781671-525-DC,way/1078532568,way/1101832649,525,1400,740934.57,f
7,8,relation/2127794-270-DC,way/23281217,way/1271288346,270,2000,69019.25,f
8,9,relation/6914309-500-DC,way/137050527,way/148382435,500,500,61383.39,f
9,10,relation/8099179-320-DC,way/1262971878,way/568527671,320,1200,161431.49,f


## Transformers

In [97]:
df_Transformers = pd.read_csv('./prerun/transformers_GB_modified.csv', header = None)
# Set the first row as header
df_Transformers.columns = df_Transformers.iloc[0]

# Drop the first row now that it’s used as header
df_Transformers = df_Transformers.drop(index=0).reset_index(drop=True)

df_Transformers.rename(columns={'transformer_id': 'TransformerID'}, inplace=True)
cols_to_keep = ['TransformerID', 'bus0', 'bus1', 'voltage_bus0', 'voltage_bus1', 's_nom']
df_Transformers = df_Transformers[cols_to_keep]
df_Transformers.to_csv('Transformers.csv', index=False)

df_Transformers

,TransformerID,bus0,bus1,voltage_bus0,voltage_bus1,s_nom
0,0,GB74-275,GB74-400,275,400,3575
1,1,relation/13110368-275,relation/13110368-400,275,400,7150
2,2,relation/14921826-275,relation/14921826-400,275,400,3575
3,3,relation/7879831-275,relation/7879831-400,275,400,7150
4,4,relation/7880111-275,relation/7880111-400,275,400,2766
...,...,...,...,...,...,...
74,74,way/94530230-275,way/94530230-400,275,400,5363
75,75,way/95749833-275,way/95749833-400,275,400,7150
76,76,way/96107758-275,way/96107758-400,275,400,7150
77,77,way/96905262-275,way/96905262-400,275,400,3575
